# 1. 특정 폴더에 패키지 설치
파이썬 패키지를 활용하기 위해서는 특정 폴더에 해당 패키지 파일을 설치하고,
해당 폴더를 사용하겠다고 선언해줘야한다.

In [ ]:
# 특정 폴더에 패키지 파일 설치 예시

# 패키지 파일을 설치할 폴더
# 띄어 쓰기가 있는 경우, cmd 에서는 전체를 쌍따옴표(")로 감싼다.
# mypath = "C:/Program Files/Python38/"
mypath = "C:/Python36"

# 주피터 노트북에서는 powershell 에서 구동되므로 띄어쓰기 있는 부분만 쌍따옴표(")로 감싼다.
# mypath = 'C:/"Program Files"/Python38/'

# ! : Terminal, CMD, Powershell 등에서 명령어를 실행한다.
# pip : 파이썬으로 작성된 패키지 소프트웨어를 설치/관리하는 패키지 관리 시스템. 파이썬 3.4 이후 버전은 pip를 기본적으로 포함함

# pip list는 현재 설치된 패키지를 확인할 수 있음
!pip list

# pip install 특정 폴더에 특정 패키지를 설치할 수 있음
# --target=$myapth 부분 없이 pip install schedule 을 하면 기본 폴더에 설치되지만, target으로 특정 폴더에 설치하고 활용하는 것을 권장함.
# !pip install --target=$mypath schedule

# pip를 업그레이드 할 수 있음
# !pip install --target=$mypath --upgrade pip

In [1]:
import time
import threading
import schedule

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta # 특정 날짜 계산하는 패키지
import pandas as pd
import numpy as np

# 
import yfinance as yf  # 미국 etf, stock 데이터
# from yahooquery import Ticker
# import yahoo_fin.stock_info as si

# 한국 주식 및 etf 
import FinanceDataReader as fdr

# 슬랙 알람 울리기
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    if response.ok:
        print("Success. Send message with slack.")
    else:
        print("Fail. Send message with slack.")
        pass

myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
# 메세지 보내기 테스트
# post_message(myToken,channel,"test_message")

In [2]:
# 휴장일 데이터 가져오기
from bs4 import BeautifulSoup
from urllib.request import urlopen

def koreaHoliday():
    mykey = apisdata["holiday"]["decoding_key"]
    
    loc_date = []
    year = "2022"
    # 1월 ~ 12월
    for i in range(1, 13):
        if i >= 10:
            month = str(i)
        elif i < 10:
            month = "0" + str(i)
        # params = {"serviceKey":mykey, "pageNo":'1', "numOfRows":"10", "solYear":year, "solMonth":month}
        params = {"serviceKey":mykey, "solYear":year, "solMonth":month}
        request_url = apisdata["holiday"]["request_url"]
        response = requests.get(request_url, params=params)
        if response.ok:
            soup = BeautifulSoup(response.text, "html.parser")
            item = soup.find_all("item")
            for i in item:
                date = i.find("locdate").get_text()
                loc_date.append(date[:4] + "-" + date[4:6] + "_" + date[6:])
        
    holiday_list = koreaHoliday()
    print("holiday_list : ", len(holiday_list))

    krx_holiday_list = ["2022-01-31",
    "2022-02-01",
    "2022-02-02",
    "2022-03-01",
    "2022-03-09",
    "2022-05-05",
    "2022-06-01",
    "2022-06-06",
    "2022-08-15",
    "2022-09-09",
    "2022-09-12",
    "2022-10-03",
    "2022-10-10",
    "2022-12-30"]
    print("krx_holiday_list : ", len(krx_holiday_list))

    holiday_list.extend(krx_holiday_list)
    print("extend_holiday_list : ", len(holiday_list))
    return holiday_list

In [3]:
# def test_function():
#     now = datetime.now()
#     print("aaa", now)

# def exit():
#     print("exit")
#     sys.exit()

# schedule.every(1).seconds.do(test_function)
# schedule.every().day.at("18:54").do(exit)

# while True:
#     schedule.run_pending()
#     time.sleep(3)



In [4]:
market_stock_dict = {
    "kospi" : None,
    "sp500" : None,
}
# kospi200 회사명 리스트 가져오기
import bs4
from urllib.request import urlopen
kospi200_list = []
for i in range(1, 21):
    page = i
    url = 'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'.format(page = page)
    source = urlopen(url).read()
    source = BeautifulSoup(source,'lxml')
    source = source.find_all('a',target = '_parent')
    for j in range(len(source)):
        name = source[j].text
        kospi200_list.append(name)

kospi_df = fdr.StockListing("kospi")
kospi_dict = kospi_df[kospi_df["Name"].isin(kospi200_list)].set_index("Symbol")["Name"].to_dict()
market_stock_dict["kospi"] = kospi_dict

In [5]:
sp500_dict = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].set_index("Symbol")["Security"].to_dict()
market_stock_dict["sp500"] = sp500_dict

In [6]:
def make_signal(df, ndays):
    try:
        df = df.reset_index()
        df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
        df["sma"] = df["TP"].rolling(ndays).mean()
        df["mad"] = df["TP"].rolling(ndays).apply(lambda x: pd.Series(x).mad())
        df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
        
        df["pre1day_cci"] = df["CCI"].shift(1)
        df["trade"] = None
        df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
        df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])
        
        df["total_buy_price"] = 0
        df["shares"] = 0
        df["buy_price"] = 0
        df["sell_price"] = 0
        df["revenue"] = 0
        df["rate"] = 0

        buy_price_list = []
        for i, x in df.iterrows():
            if x["trade"] == "buy":
                buy_price_list.append(x["Close"])
                df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                df.loc[i, "shares"] = len(buy_price_list)
                df.loc[i, "buy_price"] = np.mean(buy_price_list)
            elif x["trade"] == None:
                if i > 0:
                    df.loc[i, "shares"] = df.loc[i-1, "shares"]
                    df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                    df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
            elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                first_price = buy_price_list.pop(0)
                df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                df.loc[i, "sell_price"] = df.loc[i, "Close"]
                df.loc[i, "shares"] = len(buy_price_list)
                if len(buy_price_list) > 0:
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                else:
                    df.loc[i, "total_buy_price"] = 0
                    df.loc[i, "buy_price"] = 0
        
        r_dict = {
            "symbol" : df.iloc[-1]["Symbol"],
            "date" : df.iloc[-1]["Date"],
            "trade_signal" : df.iloc[-1]["trade"],
            "price" : df.iloc[-1]["Close"]
        }
        return r_dict
        
    except Exception as e:
        print(e)
        pass

In [7]:
# kospi200 알람 보내기

def send_trade_signal(market, symbol_dict, start_date, config_ndays):
    df = pd.DataFrame()    
    buy_name_list = []
    sell_name_list = []
    idata = []

    for symbol in symbol_dict.keys():
        try:
            if market == "kospi":
                df = fdr.DataReader(symbol, start=start_date)
            elif market == "sp500":
                df = yf.download(symbol, start=start_date, show_errors=False)
            
            df["Symbol"] = symbol
        
            r_dict = make_signal(df, config_ndays)
            
            if r_dict["trade_signal"] == "buy":
                buy_name_list.append(symbol_dict.get(r_dict["symbol"]))
                print(r_dict)
                idata.append(r_dict)
            elif r_dict["trade_signal"] == "sell":
                sell_name_list.append(symbol_dict.get(r_dict["symbol"]))
                print(r_dict)
                idata.append(r_dict)
        
        except Exception as e:
            print(e)
            pass
        
    # slack 알람 보내기

    # 구매 데이터
    if len(buy_name_list) == 0:
        post_message(myToken,channel,"today not exists buy stocks")
    if len(buy_name_list) > 0:
        post_message(myToken,channel,"buy_stocks : %s"%(buy_name_list))

    # 판매 데이터
    if len(sell_name_list) == 0:
        post_message(myToken,channel,"today not exists sell stocks")
    if len(sell_name_list) > 0:
        post_message(myToken,channel,"sell_stocks : %s"%(sell_name_list))
    
    try:
        df = pd.DataFrame.from_dict(idata)
    except Exception as e:
        print(e)
        pass
    return df

In [8]:
config_ndays = 20
start_date = (datetime.now() - timedelta(days=40)).date().strftime("%Y-%m-%d")
df = pd.DataFrame()
for key in market_stock_dict:
    r_df = send_trade_signal(market = key, symbol_dict=market_stock_dict[key], start_date=start_date, config_ndays=config_ndays)
    df.append(r_df)
    

{'symbol': '079160', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 27550}
{'symbol': '000990', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 74800}
{'symbol': '034730', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 241500}
{'symbol': '361610', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 126500}
{'symbol': '192080', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 51800}
{'symbol': '207940', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 826000}
{'symbol': '020560', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 21650}
{'symbol': '111770', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'sell', 'price': 47550}
{'symbol': '028670', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_signal': 'buy', 'price': 6990}
{'symbol': '036460', 'date': Timestamp('2022-03-31 00:00:00'), 'trade_si

In [ ]:
import pymysql
from db_config import config

conn = pymysql.connect(
    host=config['host'], 
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port'])
cur = conn.cursor(pymysql.cursors.DictCursor)

sql = """
select * from trading;
"""

cur.execute(sql)

rows = cur.fetchall()
cur.close()
conn.close()

In [ ]:
df = pd.DataFrame.from_dict(rows)
df.head()

In [ ]:
# [STEP1]한국 etf 정보 가져오기
kr_etf_df = fdr.StockListing('ETF/KR')
kr_etf_df.head()

In [ ]:
import requests
import json
from pandas import json_normalize

url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = json_normalize(json_data['result']['etfItemList'])
df.head()

In [ ]:
fdr.DataReader('069500', '2022-01-01')

In [ ]:
df.head()

In [ ]:
df['total_price'] = df['price'] * df['quantity']
g_df = df[df['action'] == 'buy'].groupby(['symbol'])[['quantity', 'total_price']].sum().reset_index()
g_df['mean_buy_price'] = g_df['total_price'] / g_df['quantity']

In [ ]:
info_columns = ['symbol', 'shortName', 'sharesOutstanding', 'floatShares', 'currentPrice', 'averageVolume', 'averageDailyVolume10Day']
idata = []
for t in g_df.symbol.unique().tolist():
    print(t, 'start')
    start_time = datetime.now()
    t_dict = dict()
    info = yf.Ticker(t).info
    for c in info_columns:
        if c not in t_dict:
            t_dict[c] = None
        if c in t_dict:
            try:
                t_dict[c] = info[c]
            except:
                try:
                    sd = Ticker(t).summary_detail
                    if c in sd[t].keys():
                        t_dict[c] = t[c]
                    else:
                        if c == 'currentPrice':
                            t_dict[c] = t['previousClose']
                        elif c == 'sharesOutstanding':
                            t_dict[c] = t['']
                except:
                    pass
    t_dict['event_datetime'] = datetime.now()
    idata.append(t_dict)
    print('end', (datetime.now() - start_time).seconds, 'seconds')
    
volume_df = pd.DataFrame(idata)
volume_df.head()

In [ ]:
volume_df.to_pickle('C:/Users/is/Downloads/volume_df_220106.pkl')

In [ ]:
v_df = pd.read_pickle('C:/Users/is/Downloads/volume_df_220106.pkl')
v_df

In [ ]:
a_df = pd.merge(g_df, v_df, how='left', on='symbol')
a_df.head()

In [ ]:
a_df['sharesReleaseRatio'] = a_df['floatShares'] / a_df['sharesOutstanding'] * 100
a_df['holding_ratio'] = a_df['quantity'] / a_df['sharesOutstanding'] * 100
a_df['rate_of_return'] = (a_df['currentPrice'] - a_df['mean_buy_price']) / a_df['mean_buy_price'] *100

In [ ]:
a_df.sort_values(['holding_ratio'], ascending=[False])

In [ ]:
from yahooquery import Ticker
t = Ticker('BNKU')

In [ ]:
t.summary_detail

In [ ]:
t.summary_detail[info_columns[3]]

In [ ]:
for row in rows:
    print(row)
    break

In [ ]:
import FinanceDataReader as fdr
df = fdr.EtfListing('KR')
df.head()

In [ ]:
df = yf.download(tickers='DVN', period='1d', interval='1m')
df.tail()

In [ ]:
market_dict = {
    'dow' : None,
    'sp500' : None,
    'nasdaq' : None
}

# 시장별 티커 리스트 가져오기
for market in market_dict:
    tickers = None

    # dow 지수 ticker list
    if market == 'dow':
        tickers = si.tickers_dow()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # sp500 지수 ticker list
    elif market == 'sp500':
        tickers = si.tickers_sp500()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # nasdaq 지수 ticker list
    elif market == 'nasdaq':
        tickers = si.tickers_nasdaq()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))


In [ ]:
yf.Tickers(market_dict['dow']).history(period='1y')['Close']

In [ ]:
# 기초 성과 지표 만들기

def create_basic_performance_indicators(market, market_ticker_list):

    # NOTE : 금융 공학에서의 1년 동안의 일수 = 252
    financial_engineering_days = 252

    i_dict = {}
    gagr = None
    mdd = None
    vol = None
    sharpe = None

    try:
        # Get tickers close price
        df = yf.Tickers(market_ticker_list).history(period='1y')['Close']
            
    except Exception as e:
        error_message = '%s error : %s'%(market, e)
        post_message(myToken, channel, error_message)

    for ticker in df.columns.tolist():
        if ticker not in i_dict:
            i_dict[ticker] = {}

        if ticker in i_dict:
            recent_date = df.index[-1]
            if recent_date not in i_dict[ticker]:
                i_dict[ticker][recent_date] = {
                    'gagr' : None,
                    'mdd' : None,
                    'vol' : None,
                    'sharpe' : None
                }

            if recent_date in i_dict[ticker]:
                ticker_df = df[ticker].reset_index()
                # 1. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률)
                # 1.1. 수정된 종가 데이터로 일별 수익률 생성하기
                ticker_df['daily_rtn'] = ticker_df[ticker].pct_change()  # 퍼센트 변화율

                # 1.2. 일별 수익률로 전략 수익률 계산하기
                ticker_df['st_rtn'] = (1 + ticker_df['daily_rtn']).cumprod()  # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.

                # 1.3. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률) 계산하기
                # NOTE : 단위 %
                gagr = (ticker_df[ticker_df['Date']==recent_date]['st_rtn'] ** (financial_engineering_days / len(ticker_df.index)) - 1) * 100

                # 2. MDD(Max Draw Down, 최대 낙폭)
                # 2.1. 관측 기간 최고점 가격 계산
                historical_max = ticker_df[ticker].cummax()  # 관측 기간 동안 누적 최댓값

                # 2.2 일별 감소비율 계산
                daily_drawdown = ticker_df[ticker] / historical_max - 1.0  # 현재 수정 종가에서 누적 최댓값 대비 낙폭률 계산

                # 2.3 일별 최대 하락률 계산
                historical_dd = daily_drawdown.cummin()  

                # 2.4 MDD 계산하기
                # NOTE : 단위 %
                mdd = - 1 * historical_dd.min() * 100

                # 3. Vol(Volatility, 변동성)
                # NOTE 단위 % 
                vol = np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days) * 100

                # 4. sharpe(ex-post Sharpe ratio, 사후적 사프 비율)
                # NOTE : 샤프 지수는 위험성 대비 수익성 지표. 사후적 샤프 비율은 자산의 실현 수익률을 사용함
                # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
                # 1 보다 크면 좋은 것으로 평가
                sharpe = np.mean(ticker_df['daily_rtn']) / np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days)

                # dictionary 에 계산 값 입력하기
                i_dict[ticker][recent_date]['gagr'] = gagr
                i_dict[ticker][recent_date]['mdd'] = mdd
                i_dict[ticker][recent_date]['vol'] = vol
                i_dict[ticker][recent_date]['sharpe'] = sharpe

    complete_message = '%s Complete basic_performance_indicators : %s'%(market, len(list(i_dict.keys())))
    post_message(myToken,channel, complete_message)
    return i_dict

In [ ]:
market = 'dow'
dow_basic_index = create_basic_performance_indicators(market, market_dict[market])

In [ ]:
market = 'sp500'
sp500_basic_index = create_basic_performance_indicators(market, market_dict[market])
sp500_basic_index

In [ ]:
def create_basic_index_dataframe(basic_index_dict):
    ilist = []
    dict = {}
    for ticker in basic_index_dict:
        try:
            recent_date = list(basic_index_dict[ticker].keys())[0]
        except Exception as e:
            print(ticker, e)
            pass
        gagr = basic_index_dict[ticker][recent_date]['gagr']
        mdd = basic_index_dict[ticker][recent_date]['mdd']
        vol = basic_index_dict[ticker][recent_date]['vol']
        sharpe = basic_index_dict[ticker][recent_date]['sharpe']
        dict = {
            'ticker' : ticker,
            'recent_date' : recent_date,
            'gagr' : gagr,
            'mdd' : mdd,
            'vol' : vol,
            'sharpe' : sharpe
        }
        
        ilist.append(dict)
    df = pd.DataFrame(ilist)
    return df

In [ ]:
sp500_df = create_basic_index_dataframe(sp500_basic_index)
sp500_df.tail()

In [ ]:
candidate_list = sp500_df[(sp500_df['sharpe'] > 1) & (sp500_df['vol'] > 40)].sort_values(by='sharpe', ascending=False)['ticker'].tolist()
print(len(candidate_list))
candidate_list

In [ ]:
def get_live_price(candidate_list):
    try:
        df = yf.download(tickers=candidate_list, period='1', interval='1m')
    except Exception as e:
        print(e)
        pass
    close_column_name = 'Adj Close'
    adjclose_df = df[close_column_name]
    for ticker in adjclose_df.columns.tolist():
        ticker_df = adjclose_df[ticker].reset_index().rename(columns={ticker:close_column_name})
        ticker_df['recent_rtn'] = ticker_df[close_column_name].pct_change()
        ticker_df['st_rtn'] = ((1 + ticker_df['recent_rtn']).cumprod() - 1) * 100   # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.
        if ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1]['st_rtn'] < -1:
            buy_message = 'Buy Now!! Ticker : %s, Recent Price : %s'%(ticker, ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1][close_column_name])
            print(buy_message)
            post_message(myToken, channel, buy_message)
            pass

In [ ]:
get_live_price(candidate_list)

In [ ]:
alb = yf.Ticker('ALB')
alb

In [ ]:
alb.info

In [ ]:
alb.actions

In [ ]:
alb.dividends

In [ ]:
alb.financials

In [ ]:
alb.major_holders

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_live_price, candidate_list)
# ticker = get_ticker()
# schedule.every(1).minutes.do(get_market_status, ticker)
# market_status, earnings_date = get_market_status(ticker)
# text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
# schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(60)
    

In [ ]:
#  특정 티커별 데이터 가져오기
def get_certain_ticker_data(certain_ticker):
    df = si.get_data(certain_ticker)
    
    # 1. 바이앤 홀드 전략
    # 수정된 종가 데이터 가져오기
    bh_df = df.loc[:, ['adjclose']].copy()

    # 수정된 종가 데이터로 일별 수익률 생성하기
    bh_df['daily_rtn'] = bh_df['adjclose'].pct_change()  # 퍼센트 변화율
    
    # 일별 수익률로 전략 수익률 계산하기
    bh_df['st_rtn'] = (1 + bh_df['daily_rtn']).cumprod()
    
    return bh_df

df = get_certain_ticker_data('AAPL')
df.head()

In [ ]:
# buy and hold strategy's returns arithmetic mean
base_date = '2021-11-01'
tmp_df = df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]
last_date = tmp_df.index[-1]
bh_ar_mean = round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
# buy and hold strategy's 
bh_ge_mean = df.loc[base_date, 'st_rtn']

In [ ]:
round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
tmp_df.loc[last_date, 'st_rtn']

In [ ]:
df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]

In [ ]:
df.loc[base_date, ['st_rtn']]

In [ ]:
price_df['daily_rtn'] = price_df['adjclose'].pct_change()
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head(10)

In [ ]:
# 1. 평균 회귀 전략
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = bb['adjclose'].rolling(n).mean()  # 중앙 이동 평균선
    bb['ub'] = bb['center'] + sigma * bb['adjclose'].rolling(n).std()  # 상단 밴드
    bb['lb'] = bb['center'] - sigma * bb['adjclose'].rolling(n).std()  # 하단 밴드
    return bb

bollinger = bollinger_band(price_df, n, sigma)
bollinger

In [ ]:
base_date = '2021-09-01'
sample = bollinger.loc[base_date:]
sample.head()

In [ ]:
book = sample[['adjclose']].copy()
book['trade'] = ''
book.head()

In [ ]:
def create_trade_book(sample):
    book = sample[['adjclose']].copy()
    book['trade'] = ''
    return book


In [ ]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'adjclose'] > sample.loc[i, 'ub']:  # 상단 밴드 이탈 시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'adjclose']:  # 하방 밴드 이탈 시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy':  # 이미 매수 상태라면
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] = 'buy'
        elif sample.loc[i, 'ub'] >=  sample.loc[i, 'adjclose'] and sample.loc[i, 'adjclose'] >= sample.loc[i, 'lb']:  # 볼린저 밴드 안에 있을 시
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] == ''  # 동작 안함
    return book


In [ ]:
book = tradings(sample, book)
book.tail(10)

In [ ]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'adjclose']
            print('진입일 : ', i, 'long 진입 가격 : ', buy)
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'adjclose']
            rtn = (sell - buy) / buy + 1 #  손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입 가격 : ', buy, 'long 청산 가격 : ', sell, '| return : ', round(rtn, 4))
        if book.loc[i, 'trade'] == '':  # 제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn =  acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc_return']  = acc_rtn
    print('Accumulated return : ', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [ ]:
returns(book)

In [ ]:
temp_df = price_df.loc[base_date:, ['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = temp_df.index[-1]
temp_df.loc[last_date, 'st_rtn']

In [ ]:
last_date

In [ ]:
base_date

In [ ]:
range = (last_date - datetime.strptime(base_date, '%Y-%m-%d')).days
CAGR = price_df.loc[base_date, 'st_rtn'] ** (range/len(price_df.loc[base_date:].index)) - 1
CAGR

In [ ]:
print(base_date, last_date)

In [ ]:
import matplotlib.pylab as plt
book['acc_return'].plot()

In [ ]:
# 2. 듀얼 모멘텀 전략
t_df = pd.DataFrame()

In [ ]:
si.get_stats(dow_list[0])

In [ ]:
t_dict = si.get_analysts_info('AMZN')
analysts_list = list(t_dict.keys())
analysts_list

In [ ]:
i = 2
print(analysts_list[i])
t_dict[analysts_list[i]]

In [ ]:
t_dict['Earnings Estimate']

In [ ]:
# slack message 보내기
import requests
from public_data_config import apisdata

def post_message(text):
    token=apisdata['slack']['token']
    channel = apisdata['slack']['channel']
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response, datetime.now())

def get_ticker():
    ticker = 'MSFT'
    return ticker

def get_market_status(ticker):
    market_status = si.get_market_status()
    earnings_date = si.get_next_earnings_date(ticker)
    return market_status, earnings_date

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_ticker)
ticker = get_ticker()
schedule.every(1).minutes.do(get_market_status, ticker)
market_status, earnings_date = get_market_status(ticker)
text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(10)
    

In [ ]:
dow_list = si.tickers_dow() ## 30개
nasdaq_list = si.tickers_nasdaq() ## 5066개
sp500_list = si.tickers_sp500()  ## 505개

"""
dow_list 와 nasdaq_list 의 중복 ticker는 APPL, AMGN, CSCO, HON, INTC, MSFT, WBA
dow_list 에 들어있는 모든 30개사는 sp500에 포함되어 있음


"""


dow_list

In [ ]:
ticker_list = dow_list + nasdaq_list + sp500_list
ticker_list - list(set(ticker_list))

# 분석 시작일

date_to = datetime.now().date() - timedelta(days=3)

# 과거 데이터 확인 기간
past_data_years = 1
delta = relativedelta(years=past_data_years)

# 데이터 시작일
date_from = date_to - delta
str_date_from = date_from.strftime('%Y-%m-%d')
str_date_to = date_to.strftime('%Y-%m-%d')
print('데이터 시작일 : ', str_date_from, str_date_to)

# 시장 종류
# 'S&P500', 'NASDAQ', 'NYSE', 'AMEX', 'SSE', 'SZSE', 'HKEX', 'TSE', 'HOSE', 'KRX'
market = 'NASDAQ'

In [ ]:
# 시장별 ticker 가져오기
market_list = ['S&P500', 'NASDAQ', 'NYSE', 'KRX']

ticker_dict = dict()
market_df = pd.DataFrame()

for i in range(len(market_list)):
    
    market = market_list[i]
    df = pd.DataFrame()
    if i == 0:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = df
        print(market_list[i])
    else:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = market_df.append(df)
        print(market_list[i])
market_df.to_pickle()

In [ ]:
market_df.groupby('Market')['Symbol'].nunique()

In [ ]:
def make_inverstment_performance_indicator(df, target_price, date):
    daily_rtn = 'daily_rnt' # 일별 수익률
    st_rtn = 'st_rtn'  # 전략 수익률 수익률 누적의 곱

    df[daily_rtn] = df[target_price].pct_change()  # 일별 수익률
    df[st_rtn] = (1 + df[daily_rtn]).cumprod()  # 전략 수익률 = 수익률의 누적 곱
    
    # CAGR = Compound Annual Growth Rate 연평균 복리 수익률, 수익률이 높을 수록 좋다.
    CAGR = df.loc[date, st_rtn] ** (252./len(df.index)) -1  # 특정 시점까지의  최종 누적 수익률의 누적 연도 제곱근, 금융공학에서 1년은 252 영업일

    # MDD = Maximum Draw Down 최대 낙폭, MDD 값이 크면 변동성이 심한 주식
    historical_max = df[target_price].cummax()
    daily_drawdown = df[target_price] / historical_max - 1.0
    historical_dd = daily_drawdown.cummin()
    MDD = historical_dd.min()

    # VOL = Volatility 휘발성, 변동성
    VOL = np.std(df[daily_rtn]) * np.sqrt(252.)

    # Sharp = ex-post Sharp ratio, 사후적 사프 비율
    # 샤프지수와 같은 공식을 사용하지만, 자산의 실현 수익률을 사용한다는 점에서 다르다.
    # 샤프지수는 위험 대비 수익성 지표를 의미함
    # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
    # 보통 1 이상이면 좋다고 판단함

    Sharp = np.mean(df[daily_rtn]) / VOL

    return CAGR, Sharp, VOL, MDD
    


In [ ]:
data = yf.Ticker('MSFT')
df = data.history(period='1d', start=str_date_from, end=str_date_to)['Close'].reset_index().set_index('Date')
df.head()

In [ ]:
df.index[-1]

In [ ]:
select_stock_dict = dict()

CAGR_criteria = 0.05 # 연평균 복리 수익률(Compound Annual Growth Rate)
Sharp_criteria = 0.01  # 샤프 지수는 0.01 이상 되면 좋다고 판단한다. (VOL을 곱하기 100 해주면 Sharp는 1)
VOL_criteria = 0.60  # 변동성 지수는 60 이상 -> 하루하루 변동성이 크면서 성장해야 단타를 자주 하지 않을까?
MDD_criteria = -0.90  # 최대 낙폭은 1년 이내에 50%까지 하락한 경우가 있으면 허용한다.
target_price = 'Close'  # 일별 종가를 기준으로 계산한다.

for ticker in ticker_dict[market_list[0]]:
    df = yf.Ticker(ticker).history(period='1y')
    # df = fdr.DataReader(ticker, str_date_from, str_date_to)[target_price].reset_index().set_index('Date')
    
    if date_to in pd.to_datetime(df.index.tolist()):
        CAGR, Sharp, VOL, MDD = make_inverstment_performance_indicator(df, target_price, str_date_to)
        # print(ticker, CAGR, Sharp, VOL, MDD)
        if CAGR >= CAGR_criteria and Sharp >= Sharp_criteria and VOL >= VOL_criteria and MDD >= MDD_criteria:
            if ticker not in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = None,
                    Sharp = None,
                    VOL = None,
                    MDD = None
                )
            if ticker in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = CAGR,
                    Sharp = Sharp,
                    VOL = VOL,
                    MDD = -1*MDD*100
                )
select_stock_dict

In [ ]:
fdr.DataReader('AAPL', '2020-01-01', '2020-01-03')

In [ ]:
import pandas as pd

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

df_listing = pd.read_html(url, header=0)[0]

cols_ren = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', 

                    '상장일':'ListingDate', '결산월':'SettleMonth',  '대표자명':'Representative', 

                    '홈페이지':'HomePage', '지역':'Region', }

df_listing = df_listing.rename(columns = cols_ren)

df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x: '{:06d}'.format(x))

df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

df_listing

In [ ]:
ms_df = fdr.DataReader(nasdaq_ticker_df.iloc[0]['Symbol'], str_start_date, str_today_date)
ms_df.head()

In [ ]:
price_df = ms_df['Close'].reset_index().set_index('Date')

In [ ]:
from_date = '2020-01-01'
to_date = '2021-01-01'
price_df.loc[from_date:to_date].plot(figsize=(16,9))

In [ ]:
# 일별 수익률
price_df['daily_rtn'] = price_df['Close'].pct_change()
price_df.head()

In [ ]:
# 바이앤 홀드 전략의 누적 곱 계산
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head()

In [ ]:
price_df['st_rtn'].plot(figsize=(16,9))

In [ ]:
base_date = '2020-04-01'
tmp_df = price_df.loc[base_date:,['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = tmp_df.index[-1]
print('누적 수익 : ', tmp_df.loc[last_date, 'st_rtn'])
tmp_df.plot(figsize=(16,9))

In [ ]:
### 투자 성과 분석 지표
# 1. 연평균 복리 수익률
# 기하 평균 N 개의 변수를 모두 곱한 값의 제곱근
# 가장 최근일의 최종 누적 수익률의 누적 연도 제곱근을 구한다. 금융공학에서 1년은 252 영업일로 계산
# ** 제곱
CAGR = price_df.loc[str_today_date, 'st_rtn'] ** (252/len(price_df.index)) - 1
CAGR

In [ ]:
# 2. 최대 낙폭(MDD Maximum Daily Drawdown)
historical_max = price_df['Close'].cummax()
daily_drawdown = price_df['Close'] / historical_max - 1.0
historical_dd = daily_drawdown.cummin()
historical_dd.plot()


In [ ]:
historical_dd[-1]

In [ ]:
# 3. 변동성
# 변동성은 금융 자산의 방향성에 대한 불확실성과 가격 등락에 대한 위험 예상 지표로 해석
# 수익률의 표준편차를 변동성으로 계산함
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
VOL

In [ ]:
# 4. 샤프 지수
# 샤프 지수는 위험 대비 수익성 지표라고 볼 수 있다.
# Ra 는 자산 수익률 Rb 는 무위험 수익률
# 사후적 샤프 비율 ex-post Sharpe ratio
# 공식은 같지만 자산의 실현 수익률을 사용한다는 점에서 샤프 지수와 다름
# 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
Sharpe = np.mean(price_df['daily_rtn']) / np.std(price_df['daily_rtn']) * np.sqrt(252.)
Sharpe

In [ ]:
import threading, time, signal
from datetime import timedelta

wait_time_seconds = 1

class ProgramKilled(Exception):
    pass

def foo():
    print(time.ctime())

def signal_handler(signum, frame):
    raise ProgramKilled

class Job(threading.Thread):
    def __init__(self, interval, execute, *args, **kwargs):
        threading.Thread.__init__(self)
        self.daemon = False
        self.stopped = threading.Event()
        self.interval = interval
        self.execute = execute
        self.args = args
        self.kwargs = kwargs
    
    def stop(self):
        self.stopped.set()
        self.join()

    def run(self):
        while not self.stopeed.wait(self.interval.total_seconds()):
            self.execute(*self.args, **self.kwargs)


if __name__ == "__main__":
    signal.signal(signal.SIGTERM, signal_handler)
    signal.signal(signal.SIGINT, signal_handler)
    job = Job(interval=timedelta(seconds=wait_time_seconds), execute=foo)
    job.start()

    while True:
        try:
            time.sleep(1)
        except ProgramKilled:
            print("Program killed: running cleanup code")
            job.stop()
            break

In [ ]:
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
post_message(myToken,channel,"test_message")